# Ранбук по теме Python p4

### Дисклеймер

Фреймворк &ndash; это такая большая библиотека, которая не только имеет набор дополнительных функций, но и указывает вам определённую структуру вашего кода. То есть, сравнивая с д/з по английскому, библиотека &ndash; это новые выученные вами слова, из которых вы составляете предложения по своему усмотрению, а фреймворк &ndash; задание "заполните пропуски" в уже готовых предложениях.

А между тем, чтобы вызвать из python код на bash и захватить результат, есть удобная библиотека:

In [ ]:
import subprocess

result = subprocess.run(["wsl", "ls", "-la"], capture_output=True).stdout.decode("utf-8")
# перед ls нужно добавить wsl, так как на windows python запускает cmd, а нам нужно выполнять команду не из-под cmd,
# а из-под линукса, параметр capture_output позволяет передать результат из bash в python, вместо stdout можно
# выбрать stderror (поток в который выводятся ошибки), а decode нужна чтобы перевести пришедшие байты в строку.
print(result)

Ну и раз уж такое дело, неплохо бы научится получать из python переменные окружения.

In [ ]:
import os

p = os.getenv('PATH')  #  получаем переменную окружения path
print(p)

## Flask

### Лирическое отступление venv

Перед установкой библиотек (а без них нам тут явно не обойтись) стоит поговорить о том, что называют virtual environment \[venv\]. Когда вы устанавливаете библиотеку модулем `pip`, вы устанавливаете её на системный интерпретатор python. Это крайне неудобно, так как этим интерпретатором пользуются все ваши приложения на компьютере, нуждающиеся в питоне. Нередко разным приложениям для работы нужна разная версия библиотек. Нетрудно видеть, что пользуясь одним лишь глобальным интерпретатором эту проблему не разрешить. Тут и вступает в игру venv. Он создаёт изолированное окружение, в которое можно устанавливать библиотеки под конкретное приложение под которое этот venv создан. Но вначале модуль venv нужно установить. Это делается спомощью команды `apt install`.

In [ ]:
%%bash
# эти команды лучше вводить из консоли, так как в них может потребоваться пароль
sudo apt update                   # обновляет список доступных системных библиотек linux
sudo apt install -y python3-venv  # скачивает модуль для python, -y означает autoyes: отвечать yes на любые
                                  #                                       вопросы, появляющиеся во время установки

In [ ]:
%%bash
# Код может выполняться пару минут

python3 -m venv .venv   # использует модуль питона (-m) под названием venv, который создаёт виртуальное окружение,
                        #                                              файлы которого будут храниться в папке .venv
                       
. .venv/bin/activate    # активирует в виртуальное окружение, теперь все действия будут происходить в нём

# Тут можно делать что-то с питоном

deactivate  # деактивирует виртуальное окружение, теперь все действия будут происходить на глобальном
                        #                                                                            интерпретаторе
                         
# На windows в PowerShell .venv\Scripts\activate.ps1

ls -a
echo "____________"
ls .venv/bin

### Api controller

Прежде чем использовать Flask (фреймворк для реализации веб сервера на python), необходимо его установить. Как и другие библиотеки python он устанавливается на компьютер спомощью модуля `pip`.

In [ ]:
import sys 
!{sys.executable} -m pip install --user --force-reinstall Flask>= 2.2.2
# {sys.executable} выдаёт путь по которому лежит python

Для простоты мы не будем использовать созданный нами venv, так как с ним нам будет сложно продолжать работу с этим jupyter notebook, но в реальной задаче, конечно, мы бы делали это из-под venv.

К слову, "!" означает, что это bash команда, а ">= 2.2.2" &ndash; требование к версии Flask. Флаг "--user" устанавливает библиотеку только для текущего пользователя, а флаг "--force-reinstall" нужен, чтобы переустановить библиотеку с нужной нам версией, если вдруг установлена старая версия библиотеки.

**Перезагрузите интерпретатор после установки библиотеки!** Выберите в верхнем меню jupyter notebook Kernel &rightarrow; Restart.

Для воспроизводимости результатов на других компьютерах, обычно делают так, записывая все необходимые библиотеки с версиями в файл requirements.txt:

In [ ]:
import sys 
!{sys.executable} -m pip freeze > requirements.txt

И на другом компьютере:

In [ ]:
import sys 
!{sys.executable} -m pip install -r requirements.txt

Теперь мы можем перейти к созланию первого api:

In [ ]:
from flask import Flask

app = Flask(__name__)  # создали веб-приложение, указав __name__, чтобы Flask умел находить файлы, лежащие 
                       #                                                                         по некоторым путям

@app.route("/")            # по корневому пути вызывать функцию index
def index():
    return "Hello World!"

Запомните код в следующей ячейке &ndash; он запускает веб-сервер и его же надо останавливать, когда в вебсервер вносятся изменения.

In [ ]:
app.run(host="0.0.0.0", port=5000)  # запускает сервер для всех в локальной сети по ip адресу машины (которое
                                     #                                           ставится на место доменного имени)
                                     #                                                                на порту 5000

Узнать свой ip можно так:

In [ ]:
import socket
socket.gethostbyname(socket.gethostname())

Теперь сайт доступен по адресу http://\<ip address>:\<port number>/ в частности по адресу http://127.0.0.1:5000/.
    
Вместо 127.0.0.1 можно поставить localhost &ndash; этот домен соответствует вышеприведённому ip.

Добавим в наш веб-сайт страниц. Возвращать можно и HTML \[что это???\]:

In [ ]:
@app.route("/htmlpage")            # по пути /htmlpage вызывать функцию get_html
def get_html():
    return "<a href=\"https://flask.palletsprojects.com/en/3.0.x/\"><img src=\"/static/rest.webp\" /></a>"

Она доступна по адресу http://127.0.0.1:5000/htmlpage.

Тут начинает играть роль то самое `__name__`, которое мы передали во `Flask`: flask автоматически распознаёт файлы, лежащие в папке static, в частности &ndash; нашу картинку.

Давайте загрузим файл с сервера.

In [ ]:
from flask import send_from_directory

@app.route("/file")
def get_file():
    return send_from_directory("./static", "rest.webp", as_attachment=True)

Но как справедливо указано на картинке REST API на то и API, что возвращает не HTML, понятный человеку, а json, понятный программам (в особенности javascriptу, ибо JSON ~ JavaScriptObjectNotation).

In [ ]:
import json

@app.route("/api")
def api_hello():
    result = {"status": 200, "msg": "This is API page"}
    return json.dumps(result)

Итого мы передали словарь! Кстати, получать данные из json тоже можно:

In [ ]:
print(json.loads('{"hello": "world", "life": "is great"}'))

Давайте теперь реализуем api, складывающее два числа.

In [ ]:
@app.route("/api/sum/<num1>/<int:num2>")  # в "<>" заключены параметры, их имена должны совпадать с именами
def api_sum(num1, num2):                  #                                                     аргументов функции
    result = {"status": 200, "data": int(num1)+num2}
    return json.dumps(result)

Заметьте, у ссылочных аргументов можно передавать тип, в таком случае, если тип не соответствует ожидаемому api выдаст 404 error (page not found). Проверить это нам поможет команда `curl` в bash, которая запрашивает сайт и возвращает содержимое. Её стоит ввести в wsl.

Для удобства, запустим сервер в фоновом режиме:

In [ ]:
import threading

thread = threading.Thread(name='server', target=lambda: app.run(host="0.0.0.0", port=5000))  
# создаёт процесс, но о них когда-нибудь в другой раз
thread.setDaemon(True)
thread.start()

**Чтобы остановить сервер прийдётся перезагружать kernel.**

In [ ]:
%%bash

curl -s http://$(hostname).local:5000/api/sum/1/2    # теперь доменное имя -- hostname компьютера
                                                     # флаг -s позволяет не выводить излишнюю информацию

echo -e "\n_________________________________________________________"

curl -s http://$(hostname).local:5000/api/sum/1/abc  # кстати, есть ещё флаг -k, который позволяет подключаться к
                                                     # небезопасным https сайтам с самоподписанным сертификатом

Другой способ передачи данных &ndash; query параметры.

In [ ]:
from flask import request

@app.route("/api/sum")
def api_sum_query():
    num1 = request.args.get('num1')
    num2 = request.args.get('num2')
    result = {"status": 200, "data": int(num1)+int(num2)}
    return json.dumps(result)

In [ ]:
%%bash

curl -s http://$(hostname).local:5000/api/sum?num1=1&num2=2

Как мы помним, имеется несколько методов для http запроса и вот как их обрабатывать:

In [ ]:
app.config['BANANAS'] = dict()                      # глобальная переменная, доступная внутри функций

@app.route("/api/banana", methods=['GET', 'POST'])  # разрешённые методы
def api_bananas():
    result = {"status": 400, "data": "smth gone wrong"}
    if request.method == 'POST':                    # создадим банан
        rdata = request.get_json(force=True)
        if "name" in rdata.keys() and "size" in rdata.keys():
            banana_name = rdata["name"]             # как мы знаем, post method позволяет прикреплять json файл
            banana_size = rdata["size"]             # к телу запроса и вот как вытащить из него данные
            app.config['BANANAS'][banana_name] = banana_size
            result = {"status": 200, "data": "created"}
        else:
            result = {"status": 400, "data": "not enough fields"}
    elif request.method == 'GET':
        result = {"status": 200, "data": app.config['BANANAS']}
    return json.dumps(result)

In [ ]:
%%bash

curl -s -X POST http://$(hostname).local:5000/api/banana -d '{"name": "ABanan","size": 15}'  # прикрепляет json
echo -e "\n_________________________________________________________"                        # документ к запросу
curl -s -X POST http://$(hostname).local:5000/api/banana -d '{"name": "BBanan", "size": 14}'
echo -e "\n_________________________________________________________"
curl -s http://$(hostname).local:5000/api/banana

И загрузим файл, для разнообразия.

In [ ]:
from werkzeug.utils import secure_filename

app.config['UPLOAD_FOLDER'] = os.getcwd()

@app.route("/api/file/<name>", methods=['POST'])
def api_upload_file(name):
    result = {"status": 400, "data": "smth gone wrong"}
    if request.method == 'POST':
        if 'file' not in request.files:
            result = {"status": 400, "data": "no file attached"}
        file = request.files['file']
        if file.filename == '':
            result = {"status": 400, "data": "no file attached"}
        if file:
            filename = secure_filename(name)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            result = {"status": 200, "data": "created"}
    return json.dumps(result)

In [ ]:
%%bash

curl -s -X POST http://$(hostname).local:5000/api/file/my_image.webp -H "Content-Type: multipart/form-data"  -F "file=@static/rest.webp"